<a href="https://colab.research.google.com/github/daybreaker95/Ampulla-classification/blob/main/collection_rate_classifier_into_two_vit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install datasets transformers accelerate torch scikit-learn matplotlib wandb

In [ ]:
# 출처: https://huggingface.co/learn/cookbook/fine_tuning_vit_custom_dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [ ]:
# !pip install openpyxl

df=pd.read_excel('/content/drive/MyDrive/박도현 교수님 연구실/췌액 채취 속도/십이지장 췌액 공유 시트 2024-11-01.xlsx', sheet_name='GI', skiprows=4)

# df는 공유 데이터셋입니다.
df = df.drop(index=0)
df.columns = df.columns.str.replace('\n', '', regex=True)

# '분'과 '초'를 추출하는 함수 정의
def extract_time(value):
    # 'x분' 형태의 데이터를 찾음
    if '분' in value and '초' not in value:
        minutes = int(value.replace('분', ''))
        return minutes
    # 'x분y초' 형태의 데이터를 찾음
    elif '분' in value and '초' in value:
        parts = value.replace('분', '').replace('초', '').split()
        minutes = int(parts[0])
        seconds = int(parts[1])
        return minutes + seconds / 60
    else:
        return None  # 해당되지 않는 경우 처리

# 채취시간 공백 또는 중도탈락인 경우는 제외함, 그리고 다시 이것을 df으로 저장함
df_drop=df[df['채취시간'].notnull()]
df_drop_2 = df_drop.drop(df_drop[df_drop['채취시간'] == '중도탈락'].index)
df=df_drop_2

#'채취시간_분 열을 생성하고, 채취시간 데이터에 extract_time 함수를 적용함, 결과값은 분 형태로 정의
df['collection_time_min'] = df['채취시간'].apply(extract_time)

# '1.6/1.1' 값을 포함한 행의 인덱스 찾기
index_to_drop = df[df['채취 용량 [cc]'] == '1.6/1.1'].index

# 해당 인덱스를 사용하여 행 제거
df = df.drop(index_to_drop)

# Calculate '채취속도_ml/min' for the remaining rows
df['collection_rate_ml/min'] = df['채취 용량 [cc]'].astype(float) / df['collection_time_min'].astype(float)
# 여기까지가 data 정리입니다.

In [ ]:
# 채취에 2분 이상 걸리는 군, 즉 채취속도 0.1ml/min 이하가 되는 군을 예측해보겠습니다.
def collection_rate_classifier(rate):
  if rate<0.1:
    return 1
  else:
    return 2

df['collection_rate_class']=df['collection_rate_ml/min'].apply(collection_rate_classifier)

In [ ]:
# 폴더 내의 모든 파일 이름 변경

import os
import pandas as pd

# 파일 이름을 변경할 폴더 경로
folder_path = '/content/drive/MyDrive/박도현 교수님 연구실/췌액 채취 속도/collection rate classification into two classes ViT/images'

# 폴더 내의 모든 파일 이름 변경
for file_name in os.listdir(folder_path):
    # 파일의 전체 경로
    old_file_path = os.path.join(folder_path, file_name)

    # 파일이 파일인지 확인
    if os.path.isfile(old_file_path):
        # 파일 이름에서 등록번호와 이름 추출
        parts = file_name.split(' ')

        try:
            # 첫 번째 부분을 등록번호로 변환 시도
            registration_number = int(parts[0])

            # df에서 해당 등록번호에 맞는 인덱스를 찾기
            matching_row = df[df['등록번호'] == registration_number]

            # 해당 등록번호가 df에 존재할 때만 진행
            if not matching_row.empty and len(parts) >= 2:
                # 이름 추출 및 collection_rate_class 값 가져오기
                name = parts[1]
                collection_rate_class = matching_row.iloc[0]['collection_rate_class']

                # 새로운 파일 이름 구성
                new_file_name = f"{registration_number}.jpg"  # 확장자에 맞게 변경
                new_file_path = os.path.join(folder_path, new_file_name)

                # 파일 이름 변경
                os.rename(old_file_path, new_file_path)
                print(f"변경됨: {old_file_path} -> {new_file_path}")
            else:
                print(f"등록번호를 찾을 수 없음 또는 이름 형식 불일치: {file_name}")

        except ValueError:
            print(f"등록번호 변환 오류: {file_name}")


등록번호 변환 오류: 33852489.jpg
등록번호 변환 오류: 62323268.jpg
등록번호 변환 오류: 35167644.jpg
등록번호 변환 오류: 68039491.jpg
등록번호 변환 오류: 69679582.jpg
등록번호 변환 오류: 69280540.jpg
등록번호 변환 오류: 68252674.jpg
등록번호 변환 오류: 60491521.jpg
등록번호 변환 오류: 55628996.jpg
등록번호 변환 오류: 68719735.jpg
등록번호 변환 오류: 69724103.jpg
등록번호 변환 오류: 67313110.jpg
등록번호 변환 오류: 68505943.jpg
등록번호 변환 오류: 69489626.jpg
등록번호 변환 오류: 69664131.jpg
등록번호 변환 오류: 69788556.jpg
등록번호 변환 오류: 24968443.jpg
등록번호 변환 오류: 11662390.jpg
등록번호 변환 오류: 69754159.jpg
등록번호 변환 오류: 44340830.jpg
등록번호 변환 오류: 69780824.jpg
등록번호 변환 오류: 69097560.jpg
등록번호 변환 오류: 68930431.jpg
등록번호 변환 오류: 69105245.jpg
등록번호 변환 오류: 63896891.jpg
등록번호 변환 오류: 69153749.jpg
등록번호 변환 오류: 69661017.jpg
등록번호 변환 오류: 13575478.jpg
등록번호 변환 오류: 69169410.jpg
등록번호 변환 오류: 69009741.jpg
등록번호 변환 오류: 19854179.jpg
등록번호 변환 오류: 62831884.jpg
등록번호 변환 오류: 68237985.jpg
등록번호 변환 오류: 67102884.jpg
등록번호 변환 오류: 69274026.jpg
등록번호 변환 오류: 63177064.jpg
등록번호 변환 오류: 69387605.jpg
등록번호 변환 오류: 50096277.jpg
등록번호 변환 오류: 42520250.jpg
등록번호 변환 오류: 69213393.jpg


In [ ]:
# images folder 내의 데이터를 'train', 'test' 폴더를 생성한 후 train_test_split으로 나누어 각각 저장해야 합니다.

# 파일을 train_test_split method를 통해 'train', 'test' 폴더에 분류합니다. stratified by 'collection_rate_class'.

from sklearn.model_selection import train_test_split
import os
import pandas as pd
import shutil

data_dir= '/content/drive/MyDrive/박도현 교수님 연구실/췌액 채취 속도/collection rate classification into two classes ViT/images'

# 데이터 디렉토리에서 파일 이름 가져오기
file_names = os.listdir(data_dir)

# 파일 이름에서 정보를 추출하고 DataFrame 생성
file_info = []
df['reg_num']=df['등록번호'].astype(int)

for file_name in file_names:
    if os.path.isfile(os.path.join(data_dir, file_name)):
        # '등록번호' '이름' 'collection_rate_class'로 구분
        parts = file_name.split('.')
        reg_num = int(parts[0])  # 확장자 제거
        collection_rate_class=df[df['reg_num']==int(reg_num)]['collection_rate_class'].values[0]
        file_info.append({'file_path': os.path.join(data_dir, file_name), 'file_name': file_name, 'reg_num': reg_num, 'collection_rate_class': collection_rate_class})

# DataFrame 생성
df_temp = pd.DataFrame(file_info)

# DataFrame 확인
print(df_temp)

# train, test 분할
train_df, test_df = train_test_split(df_temp, test_size=0.3, stratify=df_temp['collection_rate_class'], random_state=42)

# train, test 디렉토리 경로 설정
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

def copy_files(file_df, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    for _, row in file_df.iterrows():
        src_path = train_df['file_path'].values[0]
        dst_path = os.path.join(target_dir, row['file_name'])
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        shutil.copy(src_path, dst_path)

# train, val 디렉토리에 파일 복사
copy_files(train_df, train_dir)
copy_files(test_df, test_dir)

print("파일 분류가 완료되었습니다.")


                                            file_path     file_name   reg_num  \
0   /content/drive/MyDrive/박도현 교수님 연구ᄉ...  33852489.jpg  33852489   
1   /content/drive/MyDrive/박도현 교수님 연구ᄉ...  62323268.jpg  62323268   
2   /content/drive/MyDrive/박도현 교수님 연구ᄉ...  35167644.jpg  35167644   
3   /content/drive/MyDrive/박도현 교수님 연구ᄉ...  68039491.jpg  68039491   
4   /content/drive/MyDrive/박도현 교수님 연구ᄉ...  69679582.jpg  69679582   
..                                                ...           ...       ...   
72  /content/drive/MyDrive/박도현 교수님 연구ᄉ...  67848212.jpg  67848212   
73  /content/drive/MyDrive/박도현 교수님 연구ᄉ...  36964019.jpg  36964019   
74  /content/drive/MyDrive/박도현 교수님 연구ᄉ...  39681375.jpg  39681375   
75  /content/drive/MyDrive/박도현 교수님 연구ᄉ...  33087115.jpg  33087115   
76  /content/drive/MyDrive/박도현 교수님 연구ᄉ...  67251483.jpg  67251483   

    collection_rate_class  

In [ ]:
# dataset 형식에 맞추어, folder 내에 'image' 폴더를 만들고, 'labels' 라는 csv를 만드는데, A separate CSV or JSON file that maps image filenames to their corresponding labels.

labels=df_temp[['file_path', 'collection_rate_class']]
labels = labels.rename(columns={'collection_rate_class': 'label'})

# Set the file path
labels_path = '/content/drive/MyDrive/박도현 교수님 연구실/췌액 채취 속도/collection rate classification into two classes ViT/labels.csv'

# Save the DataFrame to a CSV file
labels.to_csv(labels_path, index=False)

In [ ]:
from datasets import load_dataset, ClassLabel

# Load the dataset
dataset = load_dataset('csv',
                       data_files='/content/drive/MyDrive/박도현 교수님 연구실/췌액 채취 속도/collection rate classification into two classes ViT/labels.csv')

# Convert 'label' column to ClassLabel
dataset = dataset.map(lambda x: {'label': int(x['label'])})  # Ensure labels are integers
label_classes = ClassLabel(names=list(set(dataset['train']['label'])))  # Define unique label classes
dataset = dataset.cast_column('label', label_classes)  # Cast 'label' column to ClassLabel

# Perform the train-test split with stratification
train_test_split = dataset['train'].train_test_split(test_size=0.2, stratify_by_column='label', seed=42)

# Access the train and test sets
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

print("Train Dataset:", train_dataset)
print("Test Dataset:", test_dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/77 [00:00<?, ? examples/s]

Train Dataset: Dataset({
    features: ['file_path', 'label'],
    num_rows: 61
})
Test Dataset: Dataset({
    features: ['file_path', 'label'],
    num_rows: 16
})


In [ ]:
test_dataset[0]

{'file_path': '/content/drive/MyDrive/박도현 교수님 연구실/췌액 채취 속도/collection rate classification into two classes ViT/images/68209012.jpg',
 'label': 2}

In [ ]:
# Get the numbers of each set
test_num = len(test_dataset)
train_num = len(train_dataset)

val_size = test_num / train_num

train_val_split = train_dataset.train_test_split(test_size=val_size)
train_val_split

DatasetDict({
    train: Dataset({
        features: ['file_path', 'label'],
        num_rows: 45
    })
    test: Dataset({
        features: ['file_path', 'label'],
        num_rows: 16
    })
})

In [ ]:
# train dataset에서 validation dataset을 만듭니다

from datasets import DatasetDict

dataset = DatasetDict(
    {"train": train_val_split["train"], "validation": train_val_split["test"], "test": test_dataset}
)
dataset

DatasetDict({
    train: Dataset({
        features: ['file_path', 'label'],
        num_rows: 45
    })
    validation: Dataset({
        features: ['file_path', 'label'],
        num_rows: 16
    })
    test: Dataset({
        features: ['file_path', 'label'],
        num_rows: 16
    })
})

In [ ]:
train_ds = dataset["train"]
val_ds = dataset["validation"]
test_ds = dataset["test"]

In [ ]:
train_ds[1]

{'file_path': '/content/drive/MyDrive/박도현 교수님 연구실/췌액 채취 속도/collection rate classification into two classes ViT/images/62323268.jpg',
 'label': 2}

In [ ]:
# Create mappings
id2label = {id: label for id, label in enumerate(train_ds.features["label"].names)}
label2id = {label: id for id, label in id2label.items()}

# Print the mappings
print("id2label mapping:", id2label)
print("label2id mapping:", label2id)

# Check the label for the first sample
first_sample_label = train_ds[0]["label"]
print("Label for the first sample:", first_sample_label)

# Check if the first sample label is valid
if first_sample_label in label2id:
    print("Label name for the first sample:", label2id[first_sample_label])
else:
    print(f"Label index {first_sample_label} is not in id2label mapping.")

id2label mapping: {0: 1, 1: 2}
label2id mapping: {1: 0, 2: 1}
Label for the first sample: 2
Label name for the first sample: 1


In [ ]:
train_ds.features

{'file_path': Value(dtype='string', id=None),
 'label': ClassLabel(names=[1, 2], id=None)}

In [ ]:
from transformers import ViTImageProcessor

model_name = "google/vit-large-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_name)

In [ ]:
from transformers import AutoImageProcessor

# Use an actual model name, for example:
processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")

from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip, RandomVerticalFlip,
    RandomResizedCrop,
    ToTensor,
    Resize,
)

image_mean, image_std = processor.image_mean, processor.image_std
size = (224,224) #processor.size["height"]

normalize = Normalize(mean=image_mean, std=image_std)

train_transforms = Compose(
    [
        Resize(size),
        RandomHorizontalFlip(),
        RandomVerticalFlip(),
        ToTensor(),
        normalize,
    ]
)
val_transforms = Compose(
    [
        Resize(size),
        ToTensor(),
        normalize,
    ]
)
test_transforms = Compose(
    [
        Resize(size),
        ToTensor(),
        normalize,
    ]
)

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [ ]:
from PIL import Image

def apply_train_transforms(examples):
    examples["pixel_values"] = [train_transforms(Image.open(file_path).convert("RGB")) for file_path in examples["file_path"]]
    return examples

def apply_val_transforms(examples):
    examples["pixel_values"] = [val_transforms(Image.open(file_path).convert("RGB")) for file_path in examples["file_path"]]
    return examples

def apply_test_transforms(examples):
    examples["pixel_values"] = [val_transforms(Image.open(file_path).convert("RGB")) for file_path in examples["file_path"]]
    return examples


In [ ]:
# apply transform functions to each set

train_ds.set_transform(apply_train_transforms)
val_ds.set_transform(apply_val_transforms)
test_ds.set_transform(apply_test_transforms)

In [ ]:
train_ds.features

NameError: name 'train_ds' is not defined

In [ ]:
# dataset는 Tensor으로 변환되었음을 확인합니다.

train_ds[0]

{'file_path': '/content/drive/MyDrive/박도현 교수님 연구실/췌액 채취 속도/collection rate classification into two classes ViT/images/68505943.jpg',
 'label': 2,
 'pixel_values': tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.

In [ ]:
import torch
from torch.utils.data import DataLoader

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

train_dl = DataLoader(train_ds, collate_fn=collate_fn, batch_size=4)

In [ ]:
batch = next(iter(train_dl))
for k, v in batch.items():
    if isinstance(v, torch.Tensor):
        print(k, v.shape)

pixel_values torch.Size([4, 3, 224, 224])
labels torch.Size([4])


In [ ]:
# Fine-tuning the Model

In [ ]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    model_name, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np

train_args = TrainingArguments(
    output_dir="output-models",
    save_total_limit=2,
    report_to="wandb",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=4,
    num_train_epochs=40,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="logs",
    remove_unused_columns=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model,
    train_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    tokenizer=processor,
)
trainer.train()

NameError: name 'Trainer' is not defined

In [ ]:
outputs = trainer.predict(test_ds)
print(outputs.metrics)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_true = outputs.label_ids
y_pred = outputs.predictions.argmax(1)

labels = train_ds.features["label"].names
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(xticks_rotation=45)

In [ ]:
from sklearn.metrics import recall_score

# Calculate the recall scores
# 'None' calculates recall for each class separately
recall = recall_score(y_true, y_pred, average=None)

# Print the recall for each class
for label, score in zip(labels, recall):
    print(f"Recall for {label}: {score:.2f}")